In [1]:
# vector_fields.py contains a bunch of tools to visualize vector fields and related objects
# this line is mainly needed to run the notebook in google colab or kaggle
!wget https://raw.githubusercontent.com/JeremiasE/worksheets-ode-2023/main/tools/vector_fields.py

--2023-04-17 17:33:28--  https://raw.githubusercontent.com/JeremiasE/worksheets-ode-2023/main/tools/vector_fields.py
CA-Zertifikat »/etc/ssl/certs/ca-certificates.crt« wurde geladen
Auflösen des Hostnamens raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Verbindungsaufbau zu raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 5244 (5,1K) [text/plain]
Wird in »vector_fields.py.5« gespeichert.

vector_fields.py.5  100%[===================>]   5,12K  --.-KB/s    in 0,001s  

2023-04-17 17:33:28 (7,92 MB/s) - »vector_fields.py.5« gespeichert [5244/5244]



In [ ]:
import sympy
from sympy import sin
from sympy.utilities import lambdify

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

import ipywidgets as widgets
from IPython.display import Video, YouTubeVideo

import vector_fields 


# Das mathematische Pendel

In [ ]:
(t,x,y) = sympy.var("t x y")

Das mathematische Pendel lässt sich durch die Differentialgleichung zweiter Ordnung

\begin{align*}\ddot x = -k\sin(x)\end{align*}

beschreiben. Der Parameter $k$ ist dabei gegeben durch $k= \frac{g}{\ell}$, wobei $\ell$ die Länge des Pendels und $g$
die Fallbeschleunigung ist. Im Folgenden werden wir $k$ in der Regel auf $1$ setzen.

Lösungen dieser Differentialgleichung sind Funktionen der Form $I \to \mathbb{R}$ für ein Intervall $I$. 


In [ ]:
k = 1
f = sympy.Matrix([y, -k*sin(x)])

In [ ]:
initial_value = (0.2,0)
fig = vector_fields.plot_solution_pair(f,(x,y));
fig.set_figwidth(10)
fig.set_figheight(4)
fig.axes[0].set_xlabel("$t$")
fig.axes[1].set_xlabel("$t$")
fig.axes[0].set_ylabel("$x$")
fig.axes[1].set_ylabel("$\dot x$")
fig.tight_layout()
plt.show()

## Differentialgleichungssystem und Vektorfeld

Setzen wir $y=\dot x$, so erhalten wir
das 2-dimensionale Differentialgleichungssystem
\begin{align*}\dot x &= y,\\ \dot y &= -k\sin(x).\end{align*}

Die Abbildung $\mathbb{R} \mapsto \mathbb{R},\; \begin{pmatrix}x \\ y\end{pmatrix} \mapsto \begin{pmatrix} y \\ -k\sin(x)\end{pmatrix}$ definiert ein Vektorfeld.

In [ ]:
ax = vector_fields.plot_vector_field(f,(x,-12,12),(y,-8,8))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
vector_fields.add_central_axis(ax)
plt.show()

## Einige Lösungskurven

Lösungen des obigen Differentialgleichungssystems sind Kurven im $\mathbb{R}^2$, also Abbildungen der Form $I \to \mathbb{R}^2$ für ein Intervall $I$. Die folgende Abbildung zeigt die **Bilder** einiger dieser Kurven.

In [ ]:
ax = vector_fields.plot_streamlines(f,(x,-12,12),(y,-8,8))
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
vector_fields.add_central_axis(ax)
plt.show()

Die Lösungskurven sind tangential zum Vektorfeld.

In [ ]:
ax = vector_fields.plot_vector_field(f/vector_fields.norm(f),(x,-12,12),(y,-8,8),color="black")
vector_fields.add_central_axis(ax)
vector_fields.plot_streamlines(f,(x,-12,12),(y,-8,8),ax=ax,color="tab:grey",linewidth=0.5)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
plt.show()

## Eine spezielle Lösung

Wir bestimmen nun numerisch eine Lösung der Differentialgleichung für die $x(0)=2.2, y(0)=0$ gilt.

Das Bild der Lösungskurve stellen wir nun zusammen mit dem Vektorfeld dar.

In [ ]:
def plot_solution(f, initial_value, t_1=12):
    solution = vector_fields.solve_ivp(f,(x,y),(0,t_1),initial_value)
    fig = plt.figure()
    axis = fig.subplots(1,2)
    vector_fields.plot_vector_field(f,(x,-12,12),(y,-8,8),ax=axis[0],color="black")
    vector_fields.plot_streamlines(f,(x,-12,12),(y,-8,8),ax=axis[0],color="lightgrey",linewidth=0.5)
    axis[0].plot(solution.y[0,:],solution.y[1,:],color="tab:red")
    axis[1].plot(solution.t,solution.y[0,:])
    fig.set_figwidth(16)
    fig.set_figheight(5)
    return fig

plot_solution(f, (2.2,0))
plt.show()

Wir können die Lösung auch animieren:

In [ ]:
fig = plt.figure()
fig.set_dpi(100)
#fig.set_size_inches(7, 6.5)
t_1=30
initial_value=(1.9,0)

ax = plt.axes(xlim=(0, 10), ylim=(0, 10))
patch = plt.Circle((5, -5), 0.2, color="tab:red",zorder=10)
solution = vector_fields.solve_ivp(f,(x,y),(0,t_1),initial_value)
ax.set_aspect("equal")
def init():
    vector_fields.plot_vector_field(f,(x,-12,12),(y,-8,8),ax=ax,color="black")
    vector_fields.plot_streamlines(f,(x,-12,12),(y,-8,8),ax=ax,color="lightgrey",linewidth=0.5)
    patch.center = (solution.y[0,0], solution.y[1,0])
    ax.add_patch(patch)
    return patch,

def animate(i):
    x, y = patch.center
    x = 5 + 3 * np.sin(np.radians(i))
    y = 5 + 3 * np.cos(np.radians(i))
    patch.center = (solution.y[0,i], solution.y[1,i])
    return patch,

anim = animation.FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=300, 
                               interval=20,
                               blit=True)

plt.show()


In [ ]:
# Uncomment the next line to produce the animation, this takes some time 
# anim.save("animation.mp4")
# Video("animation.mp4")

Wir können auch viele Lösungen parallel animieren (das läuft als Shader auf der GPU dank Fieldplay, https://anvaka.github.io/fieldplay/)

In [ ]:
vector_fields.field_player(f)

## Linearisierung

Die Taylorentwicklung von $\sin(x)$ in $0$ ergibt sich wie folgt:

In [ ]:
sympy.series(sin(x),x,0,10)

Für kleine Auslenkungen $x$ können wir unsere Differentialgleichung also approximieren durch $\ddot x = -kx$,
oder als System ausgedrückt, 
\begin{align*}\dot x &= y,\\ \dot y &= -kx.\end{align*}
Dieses System nennt man auch den *harmonischen Oszillator*.

Für kleine Auslenkungen sind die Lösungen noch sehr ähnlich:

In [ ]:
g = sympy.Matrix([y,-k*x])

In [ ]:
def plot_solution_with_approximation(initial_value, approximation):
    fig = plt.figure()
    axis = fig.subplots(2,2)
    solution = vector_fields.solve_ivp(f,(x,y),(0,14),initial_value)
    linearized_solution = vector_fields.solve_ivp(approximation,(x,y),(0,14),initial_value)

    vector_fields.plot_vector_field(f,(x,-6,6),(y,-4,4),ax=axis[0,0],color="black")
    vector_fields.plot_streamlines(f,(x,-6,6),(y,-4,4),ax=axis[0,0],color="lightgrey",linewidth=0.5)
    axis[0,0].plot(solution.y[0,:],solution.y[1,:],color="tab:red")
    axis[0,0].set_title("$\ddot x = - \sin(x)$")
    axis[0,1].plot(solution.t,solution.y[0,:])

    vector_fields.plot_vector_field(approximation,(x,-6,6),(y,-4,4),ax=axis[1,0],color="black")
    vector_fields.plot_streamlines(approximation,(x,-6,6),(y,-4,4),ax=axis[1,0],color="lightgrey",linewidth=0.5)
    axis[1,0].plot(linearized_solution.y[0,:],linearized_solution.y[1,:],color="tab:red")
    axis[1,0].set_title(f"$\ddot x = {sympy.latex(approximation[1])}$")
    axis[1,1].plot(linearized_solution.t,linearized_solution.y[0,:])

    fig.set_figwidth(12)
    fig.set_figheight(8)
    return fig

plot_solution_with_approximation((1,0),g)
plt.show()


Für größere Auslenkungen sind die Lösungen deutlich verschieden:

In [ ]:
plot_solution_with_approximation((2.8,0),g)
plt.show()


## Approximationen

Nimmt man eine bessere Approximation der Sinusfunktion in $0$, dann ähneln sich die Lösungen auch für größere Auslenkungen.

In [ ]:
sin_approx = sympy.series(sin(x),x,0,10).removeO()

In [ ]:
sin_approx

In [ ]:
h = sympy.Matrix([y,-k*sin_approx])

In [ ]:
plot_solution_with_approximation((2.8,0),h)
plt.show()

## Reibung

Eine Möglichkeit, die Reibung zu berücksichtigen, ist folgende Differentialgleichung:
\begin{align*}\ddot x = -k\sin(x)-c\dot{x}.\end{align*}.

In [ ]:
k = 1
c = 0.2
f2 = sympy.Matrix([y,-k*sin(x) - c*y])
plot_solution(f2,(2.2,0),30)
plt.show()

## Periodische Anregung

Fügt man jetzt noch eine periodische Anregung hinzu, erhält man z.B. folgende nicht-autonome Differentialgleichung

\begin{align*}\ddot x = -k\sin(x)-c\dot x+d \sin(t).\end{align*}

deren Lösungen deutlich wilder aussehen können.

In [ ]:
k = 1
c = 0.2
d = 6
f3 = sympy.Matrix([y,-k*sin(x)-c*y+d*sin(t)])
initial_value=(2.2,0)
time = 100
vector_fields.plot_solution_non_autonomous(f3,t,(x,y),initial_value,time)
plt.show()